# Imports

In [ ]:
%load_ext autoreload

In [ ]:
import os, sys
import holoviews as hv
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))
import hpc05

import ipywidgets as widgets
run_cluster = widgets.Checkbox(
    value=False,
    description='Check to run cluster',
    disabled=False
)
def assert_cluster_checked():
    assert run_cluster.value is True, "Command not run. Check the box above to run."

In [ ]:
import adaptive
adaptive.notebook_extension()
hv.notebook_extension('matplotlib')

import numpy as np
import scipy.constants
import cmath

import functools as ft

import sns_system, plotting_results
from distributed_sns import AggregatesSimulationSet as ASS
from distributed_sns import SimulationSet as SS

### Define constants

In [ ]:
constants = dict(
    m_eff=0.023 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

# Cluster setup

In [ ]:
run_cluster

In [ ]:
assert_cluster_checked()
hpc05.kill_remote_ipcluster()
run_cluster.value = False

In [ ]:
assert_cluster_checked()
client, dview, lview = hpc05.start_remote_and_connect(300, folder='~/two_dim_majoranas', timeout=180)
run_cluster.value = False

# Define and plot system

In [ ]:
syst_pars = {'L_down' :  1000,
               'L_up' : 1000,
               'L_m' : 500,
               'L_x' : 100,
               'a' :  12.5,
            'mu_from_bottom_of_spin_orbit_bands': True}

_=plotting_results.plot_syst(syst_pars, sns_system.dummy_params)

# Define standard parameters

In [ ]:
params_raw= dict(g_factor_middle = 10,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 1.0,
                 alpha_middle = 28,
                 alpha_left = 28,
                 alpha_right = 28,
                 Delta_left = .18,
                 Delta_right = .18,
                 B = 0.5,
                 phase = np.pi/2,
                 T = 0.0,
                 V = 0)

params = dict(**constants,
              **params_raw)

### Define keys to be varied

In [ ]:
keys_with_bounds = {"phase":[0, 2*np.pi],
                    "B":[0,1.2]}

### Define metrics to be recorded

In [ ]:
metric_params_dict = { "transparency":{},
                       "pfaffian":{}}

### Define data folder

In [ ]:
data_folder = 'spin_orbit_correction_to_chemical_potential'

# Make ASS

In [ ]:
SOI_ass = ASS(keys_with_bounds,
              syst_pars, params,
              metric_params_dict)

### Add additional dimensions

In [ ]:
alphas = np.linspace(30,60, 5)

In [ ]:
def _params_alpha(syst_pars, params, alpha):
    for key in ['alpha_middle', 'alpha_left', 'alpha_right']:
        if key in params:
            params[key] = alpha
        if key in syst_pars:
            syst_pars[key] = alpha
    return alpha

alphas = np.linspace(30, 60, 3)

params_alpha = [ft.partial(_params_alpha,
                           alpha=_alpha
                           ) for _alpha in alphas]

In [ ]:
def _params_mu_ref(syst_pars, params, mu_ref):
    syst_pars['mu_from_bottom_of_spin_orbit_bands'] = mu_ref
    return mu_ref

mu_refs = [True, False]

params_mu_ref = [ft.partial(_params_mu_ref,
                           mu_ref=_mu_ref
                           ) for _mu_ref in mu_refs]

In [ ]:
SOI_ass.add_dimension("mu_from_bottom_of_spin_orbit_bands", params_mu_ref)
SOI_ass.add_dimension("alpha", params_alpha)

### Make learner

In [ ]:
SOI_ass.make_balancing_learner(1000)
SOI_ass.load(data_folder, 1000)

### Make runner with saver

In [ ]:
runner = adaptive.Runner(SOI_ass.get_balancing_learner())
# SOI_ass.start_periodic_saver(runner, data_folder, interval=180)
runner.live_info()

In [ ]:
runner.task.result()

# Plot

In [ ]:
runner.elapsed_time()

In [ ]:
[(l.npoints, l.loss()) for l in SOI_ass.learners]


In [ ]:
%%opts Image {+framewise} [width=400 height=400 aspect=1]

kdims, plot_dict = SOI_ass.get_plot_dict(1000, False, True)

pdict = dict((k,v[:,:]) for k,v in plot_dict.items())
_=hv.HoloMap(pdict, kdims=kdims)

In [ ]:
hv.util.Dynamic(_)

In [ ]:
%%opts Image [aspect=1 colorbar=True]
%%output filename=f"./{data_folder}/spin_orbit_bottom_of_band_no_pi_fine" fig='png'
_.redim.range(z=(0,1))

In [ ]:
%%opts Image [fig_size=300 aspect=1]
_

In [ ]:
runner.cancel()